In [3]:
import os
import pandas as pd
import csv
import pre_proc_depth_id as pre

def open_csv(proc_file):
    # data = pre.data_to_proc(proc_file, proc_fields)
    # this is the consolidated csv just read each line
    data_b = os.path.join(os.getcwd(), "input", proc_file)
    return pd.read_csv(data_b)


# fields = ['date', 'lat', 'long', 'Depthm', 'T_degC', 'Salnty']
csv_data_new = open_csv("data_everything.csv")

In [6]:
df_current = csv_data_new


lat_1 = 40
lat_2 = 48
lon_1 = -135
lon_2 = -123
d_1 = 0
d_2 = 900
cond_1a = df_current.lat.between(lat_1,lat_2)
cond_1b = df_current.long.between(lon_1,lon_2)
cond_2 = df_current.date.str.startswith("194")
cond_3 = (df_current.Depthm.notnull() & df_current.T_degC.notnull()&df_current.Salnty.notnull())

cond_4 = df_current.Depthm.between(d_1,d_2)
result_1 = df_current.loc[ cond_1a & cond_1b & cond_2 & cond_3 & cond_4]



In [7]:
result_1.info()
result_1.head()

<class 'pandas.core.frame.DataFrame'>
Index: 5105 entries, 2160 to 25565
Data columns (total 6 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   date    5105 non-null   object 
 1   lat     5105 non-null   float64
 2   long    5105 non-null   float64
 3   Depthm  5105 non-null   int64  
 4   T_degC  5105 non-null   float64
 5   Salnty  5105 non-null   float64
dtypes: float64(4), int64(1), object(1)
memory usage: 279.2+ KB


,date,lat,long,Depthm,T_degC,Salnty
2160,1949-04,40.768716,-124.788358,0,10.30,33.030
2161,1949-04,40.768716,-124.788358,6,18.46,32.920
2162,1949-04,40.768716,-124.788358,10,10.29,32.951
2163,1949-04,40.768716,-124.788358,15,10.29,32.990
2164,1949-04,40.768716,-124.788358,20,10.33,33.005


In [1]:
import numpy as np
import netCDF4

def geo_idx(dd, dd_array):
    #
    #   returns the index of the closest degrees decimal value in the array.
    #   dd = degrees decimal (Float64)
    #   dd_array = arrach of degrees decimal values (Float64)
    #
    #
    geo_index = (np.abs(dd_array - dd)).argmin()
    return geo_index

def open_GEBCO_file(filepath):
    #
    #   returns dataset in NetCDF format and from the
    #   dataset arrays of the latitudes and longitudes
    #   filepath = filepath to GEBCO file
    #
    #
    NetCDF_dataset = netCDF4.Dataset(filepath)
    lats = NetCDF_dataset.variables['lat'][:]
    lons = NetCDF_dataset.variables['lon'][:]
    return NetCDF_dataset, lats, lons

def get_GEBCO_info(dataset):
    #
    #   prints metadata in GEBCO file
    #
    #
    print(dataset.data_model)

    for attr in dataset.ncattrs():
        print(attr, '=', getattr(dataset, attr))

    print(dataset.variables)
    return

def get_elevation(lat, lon):
    #
    # returns the elevation given a latitude and longitude
    #
    lat_index = geo_idx(lat, lats)
    lon_index = geo_idx(lon, lons)
    return gebco.variables['elevation'][lat_index, lon_index]

#

gebco, lats, lons  = open_GEBCO_file('GEBCO_2023_sub_ice_topo.nc')
#

In [2]:
print(get_elevation(43, -124))

92


In [44]:
from gpx_converter import Converter


# these are longitudinal ranges
# if the numbers are "neg" then they have to be swap'd
inc_end = -123
inc_start= -135


inc_step = 0.01

list_of_lats = [40, 41, 42, 43, 44,45,46,47]

lat_start = 40.6
lat_end = 47.2


def track_list(inc_start, inc_end, inc_step):
    return np.arange(inc_start, inc_end, inc_step)


def track_elevation_list( coord_list,lat_list ):
    my_dict = {}
    
    for elm_lat in lat_list:
        file_name = "track_" + str(elm_lat) + ".csv"
        gpx_file_name = "track_" + str(elm_lat) + ".gpx"
        
        with open(file_name, 'w') as fh:
            header= "lat,lon,elev\n"
            fh.write(header)
            for elm_lon in coord_list:
                elev = get_elevation(elm_lat,elm_lon)
                data_dump = str(elm_lat) + "," + str(elm_lon) + "," +str(elev)+ "\n"
                #print(data_dump)
                fh.write(data_dump)
        
        Converter(input_file=file_name).csv_to_gpx(lats_colname='lat',
                                                 longs_colname='lon',
                                                alts_colname='elev',
                                                 output_file=gpx_file_name)
                
                
                
                
                

# creates tracks based on lat numbers dumps to csv named file
c_list = track_list(inc_start, inc_end, inc_step)
track_elevation_list(c_list,list_of_lats)
    



In [5]:
import simplekml
import pandas

df=pandas.read_csv("track_41.csv")
kml=simplekml.Kml()

for lon,lat,elev in zip(df["lon"],df["lat"], df["elev"]):
    kml.newpoint(coords=[(lon,lat,elev)])

kml.save(("test.kml"))



In [4]:

import glob
import gpx2kml

gpx_list = glob.glob("*.gpx")

for each_gpx in gpx_list:
    print(each_gpx)

track_40.gpx
track_41.gpx
track_42.gpx
track_43.gpx
track_44.gpx
track_45.gpx
track_46.gpx
track_47.gpx
